In [8]:
# Import modules
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.io import SelectFiles, DataSink, FreeSurferSource
from nipype.interfaces.fsl.preprocess import FAST
from nipype.interfaces.fsl.utils import Reorient2Std
from nipype.interfaces.freesurfer import FSCommand, MRIConvert

# Set up study specific variables
#project_home = '/Volumes/iang/active/ELS/ELS_FreeSurfer/Analysis'
project_home = '/Users/lucindasisk/Dropbox/Data/fs_practice'
#project_home = '/Users/catcamacho/Dropbox/Projects/AC_ELS_Cortex/proc/fs_practice'

#fs_subjdir = '/Volumes/iang/active/ELS/ELS_FreeSurfer/ELS_FS_subjDir'
fs_subjdir = '/Users/lucindasisk/Dropbox/Data/fs_practice/ELS_FS_subjDir'
#fs_subjdir = '/Users/catcamacho/Dropbox/Projects/AC_ELS_Cortex/proc/fs_practice/ELS_FS_subjDir'

workflow_dir = project_home + '/workflows'
subj_proc = project_home + '/proc/subject'
group_proc = project_home + '/proc/group'
template_proc = project_home + '/proc/template'
subject_info = project_home + '/misc/subjects.csv' 
template_sub = ['011-T1']

#set default FreeSurfer subjects dir
FSCommand.set_default_subjects_dir(fs_subjdir)

In [9]:
######### File handling #########

#Pass in list to freesurfer source node (subs) 
fs_source = MapNode(FreeSurferSource(subjects_dir = fs_subjdir), 
                    name = 'fs_source', iterfield = ['subject_id'])
fs_source.inputs.subject_id = template_sub

#set up datasink
datasink = Node(DataSink(base_directory = template_proc),
                name = 'datasink')

180117-22:26:52,531 workflow DEBUG:
	 adding multipath trait: subject_id
180117-22:26:52,534 workflow DEBUG:
	 setting mapnode(fs_source) input: subject_id -> ['011-T1']


In [10]:
######### Template creation functions #########
def make3DTemplate(subject_T1s, num_proc, output_prefix):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    
    from os.path import abspath, split
    from os import getcwd
    from shutil import copyfile
    from glob import glob

    curr_dir = getcwd()

    #copy T1s into current directory
    for T1 in subject_T1s:
        copyfile(T1,curr_dir)

    # determine the common suffix across all the files in the subject_T1s list
    T1_suffix = subject_T1s[0]
    for a in range(0,len(subject_T1s)):
        while T1_suffix not in subject_T1s[a]:
            T1_suffix = T1_suffix[1:]
    
    # determine the common prefix across all the files in the subject_T1s list
    (folder, T1_prefix) = split(subject_T1s[0])
    for a in range(0,len(subject_T1s)):
        while T1_prefix not in subject_T1s[a]:
            T1_prefix = T1_prefix[:-1]

    # -c flag is control for local computing (2= use localhost; required for -j flag)
    # -j flag is for number of processors allowed
    call(['antsMultivariateTemplateConstruction2.sh –d 3 –o %s –r 1 –c 2 –j %d %s*%s' % (output_prefix, num_proc, T1_prefix, T1_suffix)])
    
    sample_template = abspath(output_prefix + 'template0.nii.gz')
    
    return(sample_template)


In [11]:
######### Template creation nodes #########

#convert freesurfer brainmask files to .nii
convertT1 = MapNode(MRIConvert(out_file='brainmask.nii.gz',
                               out_type='niigz'), 
                    name='convertT1', 
                    iterfield = ['in_file'])

#reorient files to standard space
reorientT1 = MapNode(Reorient2Std(),
                     name = 'reorientT1',
                     iterfield = ['in_file'])

#pass files into template function (normalized, pre-skull-stripping)
makeTemplate = Node(Function(input_names=['subject_T1s','num_proc','output_prefix'],
                             output_names=['sample_template'],
                             function=make3DTemplate), 
                    name='makeTemplate')
makeTemplate.inputs.num_proc=16 # feel free to change to suit what's free on SNI=VCS
makeTemplate.inputs.output_prefix='ELS_CT_'

180117-22:26:54,888 workflow DEBUG:
	 adding multipath trait: in_file
180117-22:26:54,895 workflow DEBUG:
	 adding multipath trait: in_file


In [12]:
######### Template creation workflow #########
template_flow = Workflow(name = "template_flow")
template_flow.connect([(fs_source, convertT1, [('T1','in_file')]),
                       (convertT1, reorientT1, [('out_file', 'in_file')]),
                       (reorientT1, makeTemplate, [('out_file', 'subject_T1s')]),
                       (makeTemplate, datasink, [('sample_template', 'sample_template')])
                      ])

template_flow.base_dir = workflow_dir
template_flow.write_graph(graph2use = 'flat')
template_flow.run()


180117-22:26:56,196 workflow DEBUG:
	 (template_flow.fs_source, template_flow.convertT1): No edge data
180117-22:26:56,198 workflow DEBUG:
	 (template_flow.fs_source, template_flow.convertT1): new edge data: {'connect': [('T1', 'in_file')]}
180117-22:26:56,200 workflow DEBUG:
	 (template_flow.convertT1, template_flow.reorientT1): No edge data
180117-22:26:56,201 workflow DEBUG:
	 (template_flow.convertT1, template_flow.reorientT1): new edge data: {'connect': [('out_file', 'in_file')]}
180117-22:26:56,202 workflow DEBUG:
	 (template_flow.reorientT1, template_flow.makeTemplate): No edge data
180117-22:26:56,204 workflow DEBUG:
	 (template_flow.reorientT1, template_flow.makeTemplate): new edge data: {'connect': [('out_file', 'subject_T1s')]}
180117-22:26:56,205 workflow DEBUG:
	 (template_flow.makeTemplate, template_flow.datasink): No edge data
180117-22:26:56,208 workflow DEBUG:
	 (template_flow.makeTemplate, template_flow.datasink): new edge data: {'connect': [('sample_template', 'sampl

180117-22:26:57,6 workflow DEBUG:
	 Needed dirs: /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/template_flow/fs_source/mapflow/_fs_source0/_report
180117-22:26:57,8 workflow DEBUG:
	 Removing files: 
180117-22:26:57,29 workflow DEBUG:
	 saved results in /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/template_flow/fs_source/mapflow/_fs_source0/result__fs_source0.pklz
180117-22:26:57,36 workflow DEBUG:
	 writing post-exec report to /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/template_flow/fs_source/mapflow/_fs_source0/_report/report.rst
180117-22:26:57,52 workflow DEBUG:
	 Finished running _fs_source0 in dir: /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/template_flow/fs_source/mapflow/_fs_source0

180117-22:26:57,129 workflow DEBUG:
	 saved results in /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/template_flow/fs_source/result_fs_source.pklz
180117-22:26:57,133 workflow DEBUG:
	 writing post-exec report to /Users/lucindasisk/Dropbox/Data/fs_pract

180117-22:27:01,478 workflow DEBUG:
	 Needed dirs: /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/template_flow/convertT1/mapflow/_convertT10/_report
180117-22:27:01,481 workflow DEBUG:
	 Removing files: 
180117-22:27:01,491 workflow DEBUG:
	 saved results in /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/template_flow/convertT1/mapflow/_convertT10/result__convertT10.pklz
180117-22:27:01,494 workflow DEBUG:
	 writing post-exec report to /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/template_flow/convertT1/mapflow/_convertT10/_report/report.rst
180117-22:27:01,502 workflow DEBUG:
	 Finished running _convertT10 in dir: /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/template_flow/convertT1/mapflow/_convertT10

180117-22:27:01,507 workflow DEBUG:
	 saved results in /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/template_flow/convertT1/result_convertT1.pklz
180117-22:27:01,510 workflow DEBUG:
	 writing post-exec report to /Users/lucindasisk/Dropbox/Data/f

180117-22:27:05,316 workflow DEBUG:
	 Needed dirs: /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/template_flow/reorientT1/mapflow/_reorientT10/_report
180117-22:27:05,318 workflow DEBUG:
	 Removing files: 
180117-22:27:05,324 workflow DEBUG:
	 saved results in /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/template_flow/reorientT1/mapflow/_reorientT10/result__reorientT10.pklz
180117-22:27:05,327 workflow DEBUG:
	 writing post-exec report to /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/template_flow/reorientT1/mapflow/_reorientT10/_report/report.rst
180117-22:27:05,332 workflow DEBUG:
	 Finished running _reorientT10 in dir: /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/template_flow/reorientT1/mapflow/_reorientT10

180117-22:27:05,342 workflow DEBUG:
	 saved results in /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/template_flow/reorientT1/result_reorientT1.pklz
180117-22:27:05,346 workflow DEBUG:
	 writing post-exec report to /Users/lucindasisk/Dr

IsADirectoryError: [Errno 21] Is a directory: '/Users/lucindasisk/Dropbox/Data/fs_practice/workflows/template_flow/makeTemplate'

In [6]:
######### Tissue creation nodes: segment template subjects to 5 tissue classes #########
### 1)CSF, 2)cortical gray matter, 3)white matter, 4)subcortical gray matter, and 5)whole brain

#convert freesurfer brainmask files to .nii
convert_to_nii = MapNode(MRIConvert(out_file='brainmask.nii.gz',
                                    out_type='niigz'), 
                         name='convert_to_nii', 
                         iterfield = ['in_file'])

#reorient files for safety :)
reorient_to_std = MapNode(Reorient2Std(),
                         name = 'reorient_to_std',
                         iterfield = ['in_file'])


#brainmask gets run through segmentation (2) ---> results in segmentation into 3 tissue classes (wm, gm, csf)
segment = MapNode(FAST(number_classes = 3, 
                       segments=True, 
                       no_bias=True), 
                  name = 'segment', 
                  iterfield = ['in_files'])

# Convert freesurfer aseg to nii
convert_aseg = MapNode(MRIConvert(out_file='aseg.nii',
                                    out_type='niigz'), 
                         name='convert_aseg', 
                         iterfield = ['in_file'])

# Reorient aseg to standard
reorient_aseg = MapNode(Reorient2Std(),
                         name = 'reorient_aseg',
                         iterfield = ['in_file'])

# Create subcortical and cortical gray matter masks <-- custom function


# make brainmask based on tissue segmentation (sc gm + cortical gm + wm) <-- custom function

180117-22:20:29,209 workflow DEBUG:
	 adding multipath trait: in_file
180117-22:20:29,213 workflow DEBUG:
	 adding multipath trait: in_file
180117-22:20:29,218 workflow DEBUG:
	 adding multipath trait: in_files
180117-22:20:29,225 workflow DEBUG:
	 adding multipath trait: in_file
180117-22:20:29,230 workflow DEBUG:
	 adding multipath trait: in_file


In [7]:
######### Tissue segmentation workflow #########
segment_flow = Workflow(name = "segment_flow")
segment_flow.connect([(fs_source, convert_to_nii, [('brainmask','in_file')]),
                      (convert_to_nii, reorient_to_std, [('out_file', 'in_file')]),
                      (reorient_to_std, segment, [('out_file', 'in_files')]),
                      (segment, convert_aseg, [('tissue_class_files', 'in_file')]),
                      (convert_aseg, reorient_aseg, [('out_file', 'in_file')]),
                      (reorient_aseg, datasink, [('out_file', 'tissue_class_files')]),
                      #(segment, datasink, [('tissue_class_files', 'tissue_class_files')])
                     ])

segment_flow.base_dir = workflow_dir
segment_flow.write_graph(graph2use = 'flat')
segment_flow.run('MultiProc', plugin_args={'n_procs': 1})

180117-22:20:34,695 workflow DEBUG:
	 (template_flow.fs_source, segment_flow.convert_to_nii): No edge data
180117-22:20:34,697 workflow DEBUG:
	 (template_flow.fs_source, segment_flow.convert_to_nii): new edge data: {'connect': [('brainmask', 'in_file')]}
180117-22:20:34,700 workflow DEBUG:
	 (segment_flow.convert_to_nii, segment_flow.reorient_to_std): No edge data
180117-22:20:34,701 workflow DEBUG:
	 (segment_flow.convert_to_nii, segment_flow.reorient_to_std): new edge data: {'connect': [('out_file', 'in_file')]}
180117-22:20:34,703 workflow DEBUG:
	 (segment_flow.reorient_to_std, segment_flow.segment): No edge data
180117-22:20:34,704 workflow DEBUG:
	 (segment_flow.reorient_to_std, segment_flow.segment): new edge data: {'connect': [('out_file', 'in_files')]}
180117-22:20:34,706 workflow DEBUG:
	 (segment_flow.segment, segment_flow.convert_aseg): No edge data
180117-22:20:34,707 workflow DEBUG:
	 (segment_flow.segment, segment_flow.convert_aseg): new edge data: {'connect': [('tissue

updatehash = False, self.overwrite = None, self._interface.always_run = True, os.path.exists(/Users/lucindasisk/Dropbox/Data/fs_practice/workflows/template_flow/fs_source/mapflow/_fs_source0/_0x0fa7661642e8da703112c5963a41c88d.json) = False, hash_method = timestamp
180117-22:20:35,913 workflow DEBUG:
	 Previous node hash = 15321c705e964fdb049e347164004d4f
180117-22:20:35,921 workflow DEBUG:
	  not presently seen: {'needed_outputs'}
180117-22:20:35,933 workflow DEBUG:
	 Removing old /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/template_flow/fs_source/mapflow/_fs_source0 and its contents
180117-22:20:35,959 workflow DEBUG:
	 Creating /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/template_flow/fs_source/mapflow/_fs_source0
180117-22:20:35,976 workflow DEBUG:
	 writing pre-exec report to /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/template_flow/fs_source/mapflow/_fs_source0/_report/report.rst
180117-22:20:35,995 workflow DEBUG:
	 Executing node
180117-22:20:36,8 

	 Needed files: /Users/lucindasisk/Dropbox/Data/fs_practice/ELS_FS_subjDir/011-T1/mri/T1.mgz;/Users/lucindasisk/Dropbox/Data/fs_practice/ELS_FS_subjDir/011-T1/mri/aseg.mgz;/Users/lucindasisk/Dropbox/Data/fs_practice/ELS_FS_subjDir/011-T1/mri/brain.mgz;/Users/lucindasisk/Dropbox/Data/fs_practice/ELS_FS_subjDir/011-T1/mri/brainmask.mgz;/Users/lucindasisk/Dropbox/Data/fs_practice/ELS_FS_subjDir/011-T1/mri/filled.mgz;/Users/lucindasisk/Dropbox/Data/fs_practice/ELS_FS_subjDir/011-T1/mri/norm.mgz;/Users/lucindasisk/Dropbox/Data/fs_practice/ELS_FS_subjDir/011-T1/mri/nu.mgz;/Users/lucindasisk/Dropbox/Data/fs_practice/ELS_FS_subjDir/011-T1/mri/orig.mgz;/Users/lucindasisk/Dropbox/Data/fs_practice/ELS_FS_subjDir/011-T1/mri/rawavg.mgz;/Users/lucindasisk/Dropbox/Data/fs_practice/ELS_FS_subjDir/011-T1/mri/rh.ribbon.mgz;/Users/lucindasisk/Dropbox/Data/fs_practice/ELS_FS_subjDir/011-T1/mri/ribbon.mgz;/Users/lucindasisk/Dropbox/Data/fs_practice/ELS_FS_subjDir/011-T1/mri/lh.ribbon.mgz;/Users/lucindasisk

180117-22:20:36,72 workflow DEBUG:
	 Needed dirs: /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/template_flow/fs_source/mapflow/_fs_source0/_report
180117-22:20:36,82 workflow DEBUG:
	 Removing files: 
180117-22:20:36,120 workflow DEBUG:
	 saved results in /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/template_flow/fs_source/mapflow/_fs_source0/result__fs_source0.pklz
180117-22:20:36,136 workflow DEBUG:
	 writing post-exec report to /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/template_flow/fs_source/mapflow/_fs_source0/_report/report.rst
180117-22:20:36,154 workflow DEBUG:
	 Finished running _fs_source0 in dir: /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/template_flow/fs_source/mapflow/_fs_source0

180117-22:20:36,204 workflow DEBUG:
	 saved results in /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/template_flow/fs_source/result_fs_source.pklz
180117-22:20:36,212 workflow DEBUG:
	 writing post-exec report to /Users/lucindasisk/Dropbox/Data/fs_

	 Needed files: /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/segment_flow/convert_to_nii/mapflow/_convert_to_nii0/brainmask.nii.gz;/Users/lucindasisk/Dropbox/Data/fs_practice/ELS_FS_subjDir/011-T1/mri/brainmask.mgz;brainmask.nii.gz;/Users/lucindasisk/Dropbox/Data/fs_practice/workflows/segment_flow/convert_to_nii/mapflow/_convert_to_nii0/_0xa37c0a8857666232317f7c0d8e5183bd_unfinished.json;/Users/lucindasisk/Dropbox/Data/fs_practice/workflows/segment_flow/convert_to_nii/mapflow/_convert_to_nii0/command.txt;/Users/lucindasisk/Dropbox/Data/fs_practice/workflows/segment_flow/convert_to_nii/mapflow/_convert_to_nii0/_inputs.pklz;/Users/lucindasisk/Dropbox/Data/fs_practice/workflows/segment_flow/convert_to_nii/mapflow/_convert_to_nii0/_node.pklz
180117-22:20:40,428 workflow DEBUG:
	 Needed dirs: /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/segment_flow/convert_to_nii/mapflow/_convert_to_nii0/_report
180117-22:20:40,442 workflow DEBUG:
	 Removing files: 
180117-22:20:40,454 wo

	 Needed files: /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/segment_flow/reorient_to_std/mapflow/_reorient_to_std0/brainmask_reoriented.nii.gz;/Users/lucindasisk/Dropbox/Data/fs_practice/workflows/segment_flow/convert_to_nii/mapflow/_convert_to_nii0/brainmask.nii.gz;/Users/lucindasisk/Dropbox/Data/fs_practice/workflows/segment_flow/reorient_to_std/mapflow/_reorient_to_std0/_0x6134e8f3705eae56f166d8b225b7ee31_unfinished.json;/Users/lucindasisk/Dropbox/Data/fs_practice/workflows/segment_flow/reorient_to_std/mapflow/_reorient_to_std0/command.txt;/Users/lucindasisk/Dropbox/Data/fs_practice/workflows/segment_flow/reorient_to_std/mapflow/_reorient_to_std0/_inputs.pklz;/Users/lucindasisk/Dropbox/Data/fs_practice/workflows/segment_flow/reorient_to_std/mapflow/_reorient_to_std0/_node.pklz
180117-22:20:44,176 workflow DEBUG:
	 Needed dirs: /Users/lucindasisk/Dropbox/Data/fs_practice/workflows/segment_flow/reorient_to_std/mapflow/_reorient_to_std0/_report
180117-22:20:44,196 workflow DE

180117-22:20:52,355 workflow DEBUG:
	 Number of pending tasks: 1
180117-22:20:54,362 workflow DEBUG:
	 MultiProcPlugin timeout before signal received. Deadlock averted??
180117-22:20:54,367 workflow DEBUG:
	 Number of pending tasks: 1
180117-22:20:56,371 workflow DEBUG:
	 MultiProcPlugin timeout before signal received. Deadlock averted??
180117-22:20:56,373 workflow DEBUG:
	 Number of pending tasks: 1
180117-22:20:58,379 workflow DEBUG:
	 MultiProcPlugin timeout before signal received. Deadlock averted??
180117-22:20:58,380 workflow DEBUG:
	 Number of pending tasks: 1
180117-22:21:00,386 workflow DEBUG:
	 MultiProcPlugin timeout before signal received. Deadlock averted??
180117-22:21:00,387 workflow DEBUG:
	 Number of pending tasks: 1
180117-22:21:02,393 workflow DEBUG:
	 MultiProcPlugin timeout before signal received. Deadlock averted??
180117-22:21:02,395 workflow DEBUG:
	 Number of pending tasks: 1
180117-22:21:04,402 workflow DEBUG:
	 MultiProcPlugin timeout before signal received.

180117-22:22:30,740 workflow DEBUG:
	 MultiProcPlugin timeout before signal received. Deadlock averted??
180117-22:22:30,742 workflow DEBUG:
	 Number of pending tasks: 1
180117-22:22:32,746 workflow DEBUG:
	 MultiProcPlugin timeout before signal received. Deadlock averted??
180117-22:22:32,748 workflow DEBUG:
	 Number of pending tasks: 1
180117-22:22:34,755 workflow DEBUG:
	 MultiProcPlugin timeout before signal received. Deadlock averted??
180117-22:22:34,757 workflow DEBUG:
	 Number of pending tasks: 1
180117-22:22:36,763 workflow DEBUG:
	 MultiProcPlugin timeout before signal received. Deadlock averted??
180117-22:22:36,764 workflow DEBUG:
	 Number of pending tasks: 1
180117-22:22:38,771 workflow DEBUG:
	 MultiProcPlugin timeout before signal received. Deadlock averted??
180117-22:22:38,773 workflow DEBUG:
	 Number of pending tasks: 1
180117-22:22:40,779 workflow DEBUG:
	 MultiProcPlugin timeout before signal received. Deadlock averted??
180117-22:22:40,780 workflow DEBUG:
	 Number 

180117-22:23:18,421 workflow DEBUG:
	 Aggregate: True
180117-22:23:18,425 workflow ERROR:
	 ['Node convert_aseg failed to run on host LSmacbook.local.']
180117-22:23:18,427 workflow INFO:
	 Saving crash info to /Users/lucindasisk/Dropbox/Github/teen_cortical/crash-20180117-222318-lucindasisk-convert_aseg-c1ca2bfd-f5ed-4f11-86d5-56b52dffefe4.pklz
180117-22:23:18,429 workflow INFO:
	 Traceback (most recent call last):
  File "/Users/lucindasisk/anaconda/lib/python3.6/site-packages/nipype/pipeline/plugins/multiproc.py", line 274, in _send_procs_to_workers
    num_subnodes = self.procs[jobid].num_subnodes()
  File "/Users/lucindasisk/anaconda/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.py", line 1234, in num_subnodes
    self._get_inputs()
  File "/Users/lucindasisk/anaconda/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.py", line 1250, in _get_inputs
    super(MapNode, self)._get_inputs()
  File "/Users/lucindasisk/anaconda/lib/python3.6/site-packages/nipype/pipelin

TypeError: 'NoneType' object is not subscriptable

In [ ]:
######### Tissue priors Nodes #########
# brainmask gets registered to template (1) ---> ---> FLIRT (fsl)-results in registration matrix and registered brainmask.nii


#apply transformation from registered brainmask to tissue segmentations ---> FLIRT (fsl)


#average each of the tissue classes together


In [ ]:
######### Tissue priors workflow #########


In [ ]:
######### Cortical thickness functions #########

cmd = 'antsCorticalThickness.sh -d 3 -a t1.nii.gz -e Template.nii.gz -m brainmask.nii.gz -p segmentationPriors%d.nii.gz -o subject'

In [ ]:
######### Cortical thickness nodes #########


In [ ]:
######### Cortical thickness workflow #########